# KGRAG Ex demo: MedMCQA (medical dataset)

This notebook demonstrates KGRAG Ex on MedMCQA, a medical multiple-choice dataset.

Workflow:
1. Load StatPearls as LangChain Documents.
2. Build the vector store and retrieval setup.
3. Build a knowledge graph from triplets extracted from the same documents.
4. Run the KGRAG Ex pipeline and convert KG paths into pseudo paragraphs.
5. Inspect explanations, optional perturbations, and report stats.



## Notebook setup and project path

Ensure the project root is on `sys.path` so local modules import correctly.


In [31]:
import sys
from pathlib import Path

project_root = next((p for p in [Path.cwd()] + list(Path.cwd().parents) if (p / "src").exists()), None)
if project_root is None:
    raise RuntimeError('"src" Verzeichnis nicht gefunden. Bitte Notebook im Projekt laufen lassen.')

root_str = str(project_root)
if root_str not in sys.path:
    sys.path.insert(0, root_str)

print("Project root:", project_root)


Project root: /Users/MeinNotebook/xai-rag


## Imports

Load data loaders, RAG components, KG builders, and the KGRAG Ex pipeline.


In [ ]:
# %%
from pathlib import Path

from src.modules.loader.statspearls_data_loader import StatPearlsDataLoader
from src.modules.loader.medmcqa_data_loader import MedMCQADataLoader

from src.modules.rag.rag_engine import RAGEngine
from src.modules.llm.llm_client import LLMClient

from src.modules.knowledge_graph.kg_triplet_extractor import KGTripletExtractor
from src.modules.knowledge_graph.kg_build_service import KGBuildService

from src.modules.knowledge_graph.kgrag_ex_pipeline import KGRAGExPipeline
from src.modules.explainers.kgrag_ex_explainer import KGRAGExExplainer


## 1. Load StatPearls, one Document per book

For a smoke test, keep only a few books and limit `max_chars_per_book`, otherwise KG building gets expensive.


In [33]:
sp_loader = StatPearlsDataLoader()

statpearls_docs, sp_stats = sp_loader.setup(
    limit_articles=300,
    as_documents=True,
    force_download=False,
    force_extract=False,
    force_rebuild_jsonl=False,
)

print("StatPearls stats:", sp_stats)
print("Docs:", len(statpearls_docs))
print("Sample meta:", statpearls_docs[0].metadata)
print("Sample preview:", statpearls_docs[0].page_content[:200])




StatPearls stats: StatPearlsBuildStats(tarball_downloaded=False, extracted=False, nxml_files_found=9629, jsonl_files_created=0, articles_loaded=300, chunks_emitted=13922)
Docs: 13922
Sample meta: {'source': 'statpearls', 'split': 'repo', 'title': 'Thrombolysis in Myocardial Infarction (TIMI) Trial, Phase I: A comparison between intravenous tissue plasminogen activator and intravenous streptokinase. Clinical findings through hospital discharge.', 'topic_name': 'Thrombolysis in Myocardial Infarction (TIMI) Trial, Phase I: A comparison between intravenous tissue plasminogen activator and intravenous streptokinase. Clinical findings through hospital discharge.', 'source_filename': 'article-100024.nxml', 'chunk_index': 0, 'chunk_id': '6de0970aebeb1d82231d62346489c3cd6ee24eebe32979badfdeae7817d4c9bb'}
Sample preview: Thrombolysis in Myocardial Infarction (TIMI) Trial, Phase I: A comparison between intravenous tissue plasminogen activator and intravenous streptokinase. Clinical findings throu

## 2. RAG engine setup

Build a fresh Chroma vector store from the documents.


In [34]:
rag = RAGEngine(persist_dir="../data/vector_db_statpearls_kgragex")
rag.setup(documents=statpearls_docs, reset=False, k_documents=6)


Loading existing vector store from ../data/vector_db_statpearls_kgragex...
RagEngine ready.


## 3. LLM client

Ollama runs locally; adjust the model name to your installation.


In [35]:
client = LLMClient(provider="ollama", model_name="llama3.2:latest")

## 4. Build the KG from StatPearls chunks (JSONL cache)

In [36]:
extractor = KGTripletExtractor(llm_client=client, max_retries=2)
kg_service = KGBuildService(extractor=extractor, add_reverse_edges=True)

kg_store, kg_stats = kg_service.build_or_load(
    docs=statpearls_docs,
    cache_path=Path("../data/statpearls_kg/kg_statpearls.jsonl"),
    limit=2000,
    force_rebuild=False,
    chunk_id_prefix="sp",
    source_name="statpearls",
)

print("KG stats:", kg_store.stats())
print("Build stats:", kg_stats)



Connecting to local Ollama (llama3.2:latest)...


KeyboardInterrupt: 

## 5. Quick KG stats


In [ ]:
import numpy as np

nodes = [str(n) for n in kg_store.g.nodes]
token_counts = np.array([len(n.split()) for n in nodes])
print("Nodes:", len(nodes))
print("Tokens per node min/median/mean/max:", token_counts.min(), np.median(token_counts), token_counts.mean(), token_counts.max())

long_nodes = [n for n in nodes if len(n.split()) > 10]
print("Nodes with >10 tokens:", len(long_nodes))
print("Example long nodes:", long_nodes[:5])

Nodes: 147
Tokens per node min/median/mean/max: 1 2.0 1.9591836734693877 7
Nodes with >10 tokens: 0
Example long nodes: []


### Graph connectivity and relation stats

Inspect connectivity, degree distribution, and relation frequencies in the KG.


In [ ]:
import networkx as nx
from collections import Counter
import numpy as np

G = kg_store.g
print("Nodes:", G.number_of_nodes(), "Edges:", G.number_of_edges())

wcc = list(nx.weakly_connected_components(G)) if G.number_of_nodes() else []
sizes = sorted([len(c) for c in wcc], reverse=True) if wcc else [0]
print("Weakly connected components:", len(wcc))
print("Largest component size:", sizes[0])

deg = np.array([d for _, d in G.degree()]) if G.number_of_nodes() else np.array([0])
print("Degree min/median/mean/max:", int(deg.min()), float(np.median(deg)), float(deg.mean()), int(deg.max()))

rel_counts = Counter()
for _, _, data in G.edges(data=True):
    rel_counts[str(data.get("relation", "Unknown"))] += 1
print("Top relations:", rel_counts.most_common(12))

Nodes: 147 Edges: 220
Weakly connected components: 39
Largest component size: 17
Degree min/median/mean/max: 2 2.0 2.993197278911565 18
Top relations: [('affects', 65), ('is_affected_by', 65), ('comorbid_with', 6), ('is_side_effect_of', 5), ('detects', 5), ('is_detected_by', 5), ('has_side_effect', 5), ('part_of', 4), ('involves_medication', 4), ('is_involved_in_treatment', 4), ('has_occurrence_of', 4), ('occurs_in', 4)]


In [ ]:
u, v, data = next(iter(G.edges(data=True)))
print("u:", u)
print("v:", v)
print("edge keys:", list(data.keys()))
print("edge data:", data)


u: CTO
v: Ischemic Heart Disease
edge keys: ['relation', 'observations', 'sources', 'chunk_ids', 'relation_conflicts']
edge data: {'relation': 'affects', 'observations': [{'relation': 'affects', 'source': 'statpearls', 'chunk_id': '7c2704e318cc7ff975ae3080b190a362122e29df4fb74c9f074d98a02c431d7e', 'meta': {'title': 'Thrombolysis in Myocardial Infarction (TIMI) Trial, Phase I: A comparison between intravenous tissue plasminogen activator and intravenous streptokinase. Clinical findings through hospital discharge.', 'question_id': '', 'source_filename': 'article-100024.nxml', 'chunk_index': 1, 'evidence': 'Continuing Education Activity Chronic total occlusion (CTO) lesions are diagnosed in patients who are undergoing coronary angiography as part of the evaluation of ischemic heart', 'label_subject': 'Body Parts', 'label_object': 'Diseases', 'relation_raw': 'AFFECTS'}}], 'sources': ['statpearls'], 'chunk_ids': ['7c2704e318cc7ff975ae3080b190a362122e29df4fb74c9f074d98a02c431d7e'], 'relation

In [ ]:
from collections import Counter

type_count = Counter(G.nodes[n].get("type", "Unknown") for n in G.nodes)
print(type_count.most_common(12))


[('BodyPart', 63), ('Disease', 43), ('Symptom', 12), ('RiskFactor', 8), ('Medication', 8), ('Procedure', 7), ('DiagnosticTest', 6)]


In [ ]:
DG = nx.DiGraph(
    (u, v, d) for u, v, d in G.edges(data=True)
    if not any(obs.get("meta", {}).get("synthetic") for obs in d.get("observations", []))
)

print(nx.is_directed_acyclic_graph(DG))

longest_path = nx.dag_longest_path(DG)
print("Longest DAG path length:", len(longest_path) - 1)
print("Path:", longest_path)


True
Longest DAG path length: 3
Path: ['Thrombosis', 'myocardial infarction', 'left circumflex artery', 'coronary artery disease']


In [ ]:
import networkx as nx

G = kg_store.g
GU = G.to_undirected()

# größte Komponente wählen
largest_comp = max(nx.connected_components(GU), key=len)
H = GU.subgraph(largest_comp).copy()

# 2x BFS Heuristik für Durchmesserpfad
n0 = next(iter(H.nodes))
u = max(nx.single_source_shortest_path_length(H, n0).items(), key=lambda x: x[1])[0]
v, dist = max(nx.single_source_shortest_path_length(H, u).items(), key=lambda x: x[1])

path = nx.shortest_path(H, u, v)

print("Largest component size:", H.number_of_nodes())
print("Approx diameter (edges):", dist)
print("Path nodes:", path)


Largest component size: 17
Approx diameter (edges): 4
Path nodes: ['Tension pneumothorax', 'Acidosis', 'Hypothermia', 'cefotaxime', 'Septicemia']


In [ ]:
path_with_rels = []
for a, b in zip(path, path[1:]):
    # da wir auf undirected arbeiten, holen wir relation aus einer Richtung, falls vorhanden
    if G.has_edge(a, b):
        rel = G[a][b].get("relation")
    elif G.has_edge(b, a):
        rel = G[b][a].get("relation")
    else:
        rel = None
    path_with_rels.append((a, rel, b))

for triple in path_with_rels:
    print(triple)


('Tension pneumothorax', 'affects', 'Acidosis')
('Acidosis', 'is_affected_by', 'Hypothermia')
('Hypothermia', 'is_side_effect_of', 'cefotaxime')
('cefotaxime', 'treats', 'Septicemia')


## 6. Pipeline and explainer


In [ ]:
pipeline = KGRAGExPipeline(rag=rag, kg=kg_store, llm_client=client)
explainer = KGRAGExExplainer(pipeline)

## 7. Load MedMCQA, small sample


In [ ]:
mcqa_loader = MedMCQADataLoader()
questions = mcqa_loader.setup(split="train", as_documents=True, limit=8)

print("MedMCQA questions:", len(questions))
print("Sample question:", questions[0].metadata.get("question"))
print("Gold:", questions[0].metadata.get("answer"))

MedMCQA questions: 8
Sample question: Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma
Gold: C


## 8. Single-run test on one MedMCQA question


In [ ]:
d = questions[0]

qid = str(d.metadata.get("question_id", "q-0"))
question_text = str(d.metadata.get("question") or d.page_content or "").strip()
gold = d.metadata.get("answer")

run = pipeline.run(
    question_id=qid,
    question=question_text,
    gold_answer=gold,
    entity_k=6,
    top_k_docs=4,
)

print("Question ID:", run.question_id)
print("Question:", run.question)
print("Gold:", run.gold_answer)
print("Entities:", run.entities)
print("Start, End:", run.start, run.end)
print("Path length:", len(run.path))
print("KG context preview:", (run.kg_context or "")[:600])
print("Answer:", run.answer)
print("LLM calls:", run.llm_calls)

None
PathAsLists(node_list=[], edge_list=[], subpath_list=[], chain_str='')

Question ID: e9ad821a-c438-4965-9f77-760819dfa155
Question: Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma
Gold: C
Entities: ['obstruction', 'hyperplasia', 'kidney']
Start, End: None None
Path length: 0
KG context preview: 
Answer: I cannot determine the change in kidney parenchyma due to chronic urethral obstruction caused by benign prostatic hyperplasia from the provided context.
LLM calls: 1


## 9. Optional: KG-level perturbations


In [ ]:
report = explainer.explain(run)

print("Most influential node:", report.most_influential_node)
print("Most influential edge:", report.most_influential_edge)
print("Most influential subpath:", report.most_influential_subpath)

print("\nFirst 10 perturbation outcomes:")
for o in report.outcomes[:10]:
    print({"kind": o.kind, "removed": o.removed, "answer_changed": o.answer_changed, "answer": (o.answer or "")[:50]})

Most influential node: None
Most influential edge: None
Most influential subpath: None

First 10 perturbation outcomes:


## 10. Mini-batch test


In [ ]:
import pandas as pd

rows = []
for d in questions:
    qid = str(d.metadata.get("question_id", "q"))
    qtxt = str(d.metadata.get("question") or d.page_content or "").strip()
    gold = d.metadata.get("answer")

    run_i = pipeline.run(
        question_id=qid,
        question=qtxt,
        gold_answer=gold,
        entity_k=6,
        top_k_docs=4,
    )
    rep_i = explainer.explain(run_i)
    num_changed = sum(1 for o in rep_i.outcomes if o.answer_changed)

    rows.append(
        {
            "question_id": run_i.question_id,
            "gold": run_i.gold_answer,
            "path_len": len(run_i.path),
            "entities": ", ".join(run_i.entities[:6]),
            "answer": (run_i.answer or "")[:10],
            "perturbations": len(rep_i.outcomes),
            "answer_changes": num_changed,
            "most_node": rep_i.most_influential_node,
            "most_edge": rep_i.most_influential_edge,
            "most_subpath": rep_i.most_influential_subpath,
        }
    )

df = pd.DataFrame(rows)
df

None
PathAsLists(node_list=[], edge_list=[], subpath_list=[], chain_str='')

None
PathAsLists(node_list=[], edge_list=[], subpath_list=[], chain_str='')

None
PathAsLists(node_list=[], edge_list=[], subpath_list=[], chain_str='')

('Optic Nerve Dysfunction', 'left circumflex artery')
PathAsLists(node_list=[], edge_list=[], subpath_list=[], chain_str='')

None
PathAsLists(node_list=[], edge_list=[], subpath_list=[], chain_str='')

None
PathAsLists(node_list=[], edge_list=[], subpath_list=[], chain_str='')

None
PathAsLists(node_list=[], edge_list=[], subpath_list=[], chain_str='')

None
PathAsLists(node_list=[], edge_list=[], subpath_list=[], chain_str='')



,question_id,gold,path_len,entities,answer,perturbations,answer_changes,most_node,most_edge,most_subpath
0,e9ad821a-c438-4965-9f77-760819dfa155,C,0,"obstruction, hyperplasia, kidney",I cannot d,0,0,None,None,None
1,e3d3c4e1-4fb2-45e7-9f88-247cc8f373b3,C,0,"Vitamin D, Vitamin B12, Vitamin A",A,0,0,None,None,None
2,5c38bea6-787a-44a9-b2df-88f4218ab914,D,0,"surgical, options, for, morbid, obesity, except",C,0,0,None,None,None
3,cdeedb04-fbe9-432c-937c-d53ac24475de,A,0,"common carotid, optic nerve, artery, thrombus,...",C,0,0,None,None,None
4,dc6794a3-b108-47c5-8b1b-3b4931577249,B,0,"growth, hormone, effect, on, growth, through",A,0,0,None,None,None
5,5ab84ea8-12d1-47d4-ab22-668ebf01e64c,C,0,"Scrub typhus, typhus, scrub",I cannot d,0,0,None,None,None
6,a83de6e4-9427-4480-b404-d96621ebb640,C,0,"colposcopy, cervical intraepithelial neoplasia...",I cannot d,0,0,None,None,None
7,f3bf8583-231b-4b7a-828c-179b0f9ccdd9,C,0,"Rectal exam, Anal fissure, Hemorrhoids, Divert...",A,0,0,None,None,None


## 11. Optional: compact stats for your RQs

RQ2 (position proxy) depends on explainer implementation details.  
RQ3 (node types) depends on which entity types your triplet extractor emits.  
RQ4 (graph metrics) depends on whether your metrics module computes degree and betweenness.

If your report exposes these fields, we summarize them here.


In [ ]:
if getattr(report, "rq2_positions", None):
    print("RQ2 positions sample:", report.rq2_positions[:5])

if getattr(report, "rq3_node_types", None):
    print("RQ3 node types sample:", report.rq3_node_types[:10])

if getattr(report, "rq4_graph_metrics", None):
    print("RQ4 graph metrics sample:", report.rq4_graph_metrics[:5])


## 12. Try your own question

Use this to quickly check whether the KG has enough coverage.

Note:
If `path_len` is often 0, it's usually an entity extraction/matching or KG coverage issue, not a RAG issue.


In [ ]:
qid = "manual-1"
question = "How does CTO relates to Cardiomyopathy"

run_m = pipeline.run(
    question_id=qid,
    question=question,
    gold_answer=None,
    entity_k=6,
    top_k_docs=4,
)

print("Entities:", run_m.entities)
print("Start, End:", run_m.start, run_m.end)
print("Path length:", len(run_m.path))
print("KG context preview:", (run_m.kg_context or "")[:400])
print("Answer:", run_m.answer)

rep_m = explainer.explain(run_m)
print("Most influential node:", rep_m.most_influential_node)
print("Most influential edge:", rep_m.most_influential_edge)
print("Most influential subpath:", rep_m.most_influential_subpath)


('CTO', 'Cardiomyopathy')
PathAsLists(node_list=['CTO', 'Ischemic Heart Disease', 'Cardiomyopathy'], edge_list=['affects', 'comorbid_with'], subpath_list=[('CTO', 'affects', 'Ischemic Heart Disease'), ('Ischemic Heart Disease', 'comorbid_with', 'Cardiomyopathy')], chain_str='CTO->[affects]->Ischemic Heart Disease->[comorbid_with]->Cardiomyopathy')
CTO->[affects]->Ischemic Heart Disease->[comorbid_with]->Cardiomyopathy
Entities: ['CTO', 'Cardiomyopathy']
Start, End: CTO Cardiomyopathy
Path length: 2
KG context preview: A cerebrovascular accident (CTO) can have a significant impact on cardiovascular health, as it often leads to ischemia in the brain. Ischemic heart disease, characterized by reduced blood flow to the heart muscle, is a common complication of CTO and shares many risk factors with cardiomyopathy, a condition where the heart muscle becomes weakened and cannot function properly. In fact, individuals w
Answer: CTO relates to Cardiomyopathy as a risk factor and complication.
Mo

In [ ]:
qid = "manual-1"
question = "How does Tension pneumothorax relates to Septicemia"

run_m = pipeline.run(
    question_id=qid,
    question=question,
    gold_answer=None,
    entity_k=6,
    top_k_docs=4,
)

print("Entities:", run_m.entities)
print("Start, End:", run_m.start, run_m.end)
print("Path length:", len(run_m.path))
print("KG context preview:", (run_m.kg_context or "")[:400])
print("Answer:", run_m.answer)

rep_m = explainer.explain(run_m)
print("Most influential node:", rep_m.most_influential_node)
print("Most influential edge:", rep_m.most_influential_edge)
print("Most influential subpath:", rep_m.most_influential_subpath)


('Tension pneumothorax', 'Septicemia')
PathAsLists(node_list=['Tension pneumothorax', 'Acidosis', 'Hypothermia', 'cefotaxime', 'Septicemia'], edge_list=['affects', 'is_affected_by', 'is_side_effect_of', 'treats'], subpath_list=[('Tension pneumothorax', 'affects', 'Acidosis'), ('Acidosis', 'is_affected_by', 'Hypothermia'), ('Hypothermia', 'is_side_effect_of', 'cefotaxime'), ('cefotaxime', 'treats', 'Septicemia')], chain_str='Tension pneumothorax->[affects]->Acidosis->[is_affected_by]->Hypothermia->[is_side_effect_of]->cefotaxime->[treats]->Septicemia')
Tension pneumothorax->[affects]->Acidosis->[is_affected_by]->Hypothermia->[is_side_effect_of]->cefotaxime->[treats]->Septicemia
Entities: ['Tension', 'pneumothorax', 'septicemia']
Start, End: Tension pneumothorax Septicemia
Path length: 4
KG context preview: A tension pneumothorax is a life-threatening condition characterized by the accumulation of air in the pleural space, leading to increased intrathoracic pressure. This can cause shift

### Optional: print RQ metrics for the manual question

Surface the RQ fields directly for quick inspection.


In [ ]:
print("sensitivity:", rep_m.rq1_sensitivity)
print("Postion:", rep_m.rq2_positions)
print("Node types:", rep_m.rq3_node_types)
print("graph_metrics:", rep_m.rq4_graph_metrics)

sensitivity: RQ1Sensitivity(node_changed=2, edge_changed=3, subpath_changed=3, node_total=5, edge_total=4, subpath_total=4)
Postion: [RQ2PositionRecord(kind='node', removed='Tension pneumothorax', index=0, rel_pos=0.0, answer_changed=True), RQ2PositionRecord(kind='node', removed='Acidosis', index=1, rel_pos=0.25, answer_changed=False), RQ2PositionRecord(kind='node', removed='Hypothermia', index=2, rel_pos=0.5, answer_changed=False), RQ2PositionRecord(kind='node', removed='cefotaxime', index=3, rel_pos=0.75, answer_changed=True), RQ2PositionRecord(kind='node', removed='Septicemia', index=4, rel_pos=1.0, answer_changed=False), RQ2PositionRecord(kind='edge', removed='Tension pneumothorax::affects::Acidosis', index=0, rel_pos=0.0, answer_changed=True), RQ2PositionRecord(kind='edge', removed='Acidosis::is_affected_by::Hypothermia', index=1, rel_pos=0.3333333333333333, answer_changed=True), RQ2PositionRecord(kind='edge', removed='Hypothermia::is_side_effect_of::cefotaxime', index=2, rel_pos=

### Debug: graph connectivity between start/end nodes

Check whether the start/end nodes exist, inspect neighbors, and probe shortest paths.


In [ ]:
import networkx as nx

G = kg_store.g
start = run_m.start
end = run_m.end

print("start exists:", start in G)
print("end exists:", end in G)

if start in G:
    print("\nStart neighbors (out):", list(G.successors(start))[:30])
    print("Start neighbors (in):", list(G.predecessors(start))[:30])

if end in G:
    print("\nEnd neighbors (out):", list(G.successors(end))[:30])
    print("End neighbors (in):", list(G.predecessors(end))[:30])

if start in G and end in G:
    GU = G.to_undirected(as_view=True)
    try:
        p = nx.shortest_path(GU, start, end)
        print("\nUndirected shortest path length:", len(p) - 1)
        print("Path nodes:", p[:20], "..." if len(p) > 20 else "")
    except Exception as e:
        print("\nNo undirected path:", e)

# Additional: search for similar node names
def find_similar_nodes(q, limit=20):
    ql = (q or "").lower()
    hits = [n for n in G.nodes if ql in str(n).lower()]
    return hits[:limit]

print("\nSimilar to 'CTO':", find_similar_nodes("CTO", 30))
print("Similar to 'Chronic Total Occlusion':", find_similar_nodes("Chronic Total Occlusion", 30))
print("Similar to 'Stroke':", find_similar_nodes("Stroke", 30))
print("Similar to 'Ischemic Heart Disease':", find_similar_nodes("Ischemic Heart Disease", 30))


start exists: True
end exists: True

Start neighbors (out): ['Acidosis']
Start neighbors (in): ['Acidosis']

End neighbors (out): ['cefotaxime']
End neighbors (in): ['cefotaxime']

Undirected shortest path length: 4
Path nodes: ['Tension pneumothorax', 'Acidosis', 'Hypothermia', 'cefotaxime', 'Septicemia'] 

Similar to 'CTO': ['CTO', 'CTO vessel', 'CTO PCI']
Similar to 'Chronic Total Occlusion': ['Chronic Total Occlusion']
Similar to 'Stroke': []
Similar to 'Ischemic Heart Disease': ['Ischemic Heart Disease']
